<a href="https://colab.research.google.com/github/YuriArduino/Estudos_Artificial_Intelligence/blob/llamaIndex-analisando-dados-pandas/pandas_workflow_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas Workflow com LlamaIndex Workflows

##Instalar Dependências:

In [7]:
# Instalar dependências (-q para instalação silenciosa)
!pip install -q llama-index llama-index-llms-openai llama-index-llms-groq llama-index-experimental gradio fpdf


# Sobre a Atualização do LlamaIndex

O **LlamaIndex** descontinuou o módulo **QueryPipeline** em favor de uma nova abordagem chamada **Workflows**.
Essa mudança foi implementada na versão `0.11` do LlamaIndex, com o objetivo de oferecer uma arquitetura mais
flexível e escalável para a construção de aplicações de IA generativa.

### Por que o QueryPipeline foi descontinuado?
O QueryPipeline era uma API declarativa útil para orquestrar consultas simples a avançadas sobre os dados.
Porém, ele se mostrou limitado para cenários mais dinâmicos. Por isso, os **Workflows** foram introduzidos,
trazendo uma arquitetura orientada a eventos para fluxos de trabalho mais sofisticados.

### O que são Workflows?
Workflows permitem orquestrar etapas personalizadas de forma assíncrona e condicional, facilitando
integrações complexas e manipulação de dados em tempo real.

**Documentação oficial:** [docs.llamaindex.ai](https://docs.llamaindex.ai)


#Código:


##Imports

In [8]:
import pandas as pd
import textwrap
from pydantic import BaseModel, Field, field_validator, SkipValidation
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.workflow import Workflow, Event, StartEvent, StopEvent, step
from google.colab import userdata
import re

## ===== Configuração Pydantic V2 =====

In [9]:
class LLMConfig(BaseModel):
    model: str = Field(..., description="Nome do modelo Groq a ser usado")
    api_key: str = Field(..., description="Chave da API Groq")
    data_url: str = Field(..., description="URL do CSV com os dados")

    @field_validator("data_url")
    @classmethod
    def validar_url(cls, v: str) -> str:
        if not (v.startswith("http://") or v.startswith("https://")):
            raise ValueError("data_url deve começar com http:// ou https://")
        return v

    @field_validator("api_key")
    @classmethod
    def validar_api_key(cls, v: str) -> str:
        if not v or len(v.strip()) == 0:
            raise ValueError("api_key não pode ser vazia")
        return v

##Funções Textuais:

In [11]:
def formatar_texto(response: str, largura: int = 100, imprimir: bool = True):
    texto_formatado = textwrap.fill(response, width=largura)
    if imprimir:
        print(texto_formatado)
    return texto_formatado

def descricao_colunas(df: pd.DataFrame) -> str:
    descricao = "\n".join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Colunas do DataFrame:\n" + descricao

def limpar_codigo_pandas(codigo: str) -> str:
    codigo = re.sub(r'```(?:python)?\n?', '', codigo)
    linhas = codigo.split('\n')
    codigo_limpo = []
    for linha in linhas:
        linha = linha.strip()
        if (linha and
            not linha.startswith('#') and
            not linha.startswith('//') and
            not linha.lower().startswith('resposta:') and
            not linha.lower().startswith('resultado:') and
            not linha.lower().startswith('código:')):
            codigo_limpo.append(linha)
    codigo_final = '\n'.join(codigo_limpo).strip()
    if '\n' in codigo_final:
        for linha in codigo_final.split('\n'):
            if linha.startswith('df.') or linha.startswith('df['):
                return linha
    return codigo_final

### Prompts:

In [13]:
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função eval().\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO.\n"
    "5. Não coloque a expressão entre aspas.\n"
)

pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado df.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de print(df.head()):\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

response_synthesis_prompt_str = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas:\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta:"
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

In [14]:
key = userdata.get("Groq_API")

config = LLMConfig(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key=key,
    data_url="https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
)

df = pd.read_csv(config.data_url)
Settings.llm = Groq(model=config.model, api_key=config.api_key)

print("Estrutura do DataFrame:")
print(df.head())
print("\nColunas disponíveis:")
print(df.columns.tolist())
print("\nInfo do DataFrame:")
print(df.info())

Estrutura do DataFrame:
     ID_compra filial       cidade tipo_cliente     genero       tipo_produto  \
0  750-67-8428      A  Santo André       Membro   Feminino     Saúde e Beleza   
1  226-31-3081      C  São Caetano       Normal   Feminino        Eletrônicos   
2  631-41-3108      A  Santo André       Normal  Masculino               Casa   
3  123-19-1176      A  Santo André       Membro  Masculino     Saúde e Beleza   
4  373-73-7910      A  Santo André       Normal  Masculino  Esportes e Viagem   

   preco_unitario  quantidade  imposto_5%     total        data      hora  \
0           74.69           7     26.1415  548.9715  2024-01-05  13:08:00   
1           15.28           5      3.8200   80.2200  2024-03-08  10:29:00   
2           46.33           7     16.2155  340.5255  2024-03-03  13:23:00   
3           58.22           8     23.2880  489.0480  2024-01-27  20:33:00   
4           86.31           7     30.2085  634.3785  2024-02-08  10:37:00   

     forma_pagamento  aval

In [15]:
class CodeEvent(Event):
    pandas_prompt: str
    query: str

class OutputEvent(Event):
    pandas_code: str
    pandas_output: SkipValidation
    query: str
    pandas_prompt: str

class ExecutedEvent(OutputEvent):
    pass

class ResponseEvent(StopEvent):
    query: str
    pandas_code: str
    pandas_output: SkipValidation
    resposta_final: str

class PandasWorkflow(Workflow):
    llm = Settings.llm
    workflow_steps = ["start_query", "generate_code", "execute_code", "synthesize_response"]

    @step
    async def start_query(self, ev: StartEvent) -> CodeEvent:
        colunas_info = descricao_colunas(df)
        prompt_text = pandas_prompt_str.format(
            colunas_detalhes=colunas_info,
            df_str=df.head(),
            instruction_str=instruction_str,
            query_str=ev.query
        )
        return CodeEvent(pandas_prompt=prompt_text, query=ev.query)

    @step
    async def generate_code(self, ev: CodeEvent) -> OutputEvent:
        response = await self.llm.acomplete(ev.pandas_prompt)
        codigo_limpo = limpar_codigo_pandas(str(response).strip())
        print(f"Código gerado: {codigo_limpo}")
        return OutputEvent(
            pandas_code=codigo_limpo,
            pandas_output=None,
            query=ev.query,
            pandas_prompt=ev.pandas_prompt
        )

    @step
    async def execute_code(self, ev: OutputEvent) -> ExecutedEvent:
        try:
            if not ev.pandas_code or not ev.pandas_code.strip():
                resultado = "Erro: Código vazio gerado"
            elif not ev.pandas_code.startswith('df'):
                resultado = f"Erro: Código deve começar com 'df', recebido: {ev.pandas_code}"
            else:
                resultado = eval(ev.pandas_code, {"df": df, "pd": pd})
                print(f"Resultado da execução: {resultado}")
        except Exception as e:
            resultado = f"Erro ao executar '{ev.pandas_code}': {str(e)}"
            print(f"Erro na execução: {resultado}")
        return ExecutedEvent(
            pandas_code=ev.pandas_code,
            pandas_output=resultado,
            query=ev.query,
            pandas_prompt=ev.pandas_prompt
        )

    @step
    async def synthesize_response(self, ev: ExecutedEvent) -> ResponseEvent:
        if isinstance(ev.pandas_output, str) and ev.pandas_output.startswith("Erro"):
            resposta_final = f"Não foi possível processar a consulta: {ev.pandas_output}"
        else:
            prompt_text = response_synthesis_prompt_str.format(
                query_str=ev.query,
                pandas_instructions=ev.pandas_code,
                pandas_output=ev.pandas_output
            )
            resposta = await self.llm.acomplete(prompt_text)
            resposta_final = str(resposta).strip()
        return ResponseEvent(
            query=ev.query,
            pandas_code=ev.pandas_code,
            pandas_output=ev.pandas_output,
            resposta_final=resposta_final
        )

In [16]:
import asyncio

async def executar_consulta(query: str):
    wf = PandasWorkflow(timeout=60, verbose=False)
    try:
        result = await wf.run(query=query)
        print(f"\n=== RESULTADO FINAL ===")
        print(f"Query: {result.query}")
        print(f"Código: {result.pandas_code}")
        print(f"Resultado: {result.pandas_output}")
        print(f"\n=== EXPLICAÇÃO ===")
        formatar_texto(result.resposta_final)
        return result
    except Exception as e:
        print(f"Erro no workflow: {str(e)}")
        return None

# Executar exemplo
query = "Qual é a avaliação média de cada filial?"
await executar_consulta(query)

Código gerado: df.groupby('filial')['avaliacao'].mean().reset_index().set_index('filial').T
Resultado da execução: filial            A         B         C
avaliacao  7.027059  6.818072  7.072866

=== RESULTADO FINAL ===
Query: Qual é a avaliação média de cada filial?
Código: df.groupby('filial')['avaliacao'].mean().reset_index().set_index('filial').T
Resultado: filial            A         B         C
avaliacao  7.027059  6.818072  7.072866

=== EXPLICAÇÃO ===
Com base nos resultados fornecidos pela consulta, podemos observar que a avaliação média de cada
filial é a seguinte:  - Filial A: 7,027059 - Filial B: 6,818072 - Filial C: 7,072866  Esses valores
indicam que a Filial C tem a avaliação média mais alta, seguida de perto pela Filial A, e a Filial B
tem a avaliação média mais baixa entre as três.  O código utilizado foi
`df.groupby('filial')['avaliacao'].mean().reset_index().set_index('filial').T`


ResponseEvent(query='Qual é a avaliação média de cada filial?', pandas_code="df.groupby('filial')['avaliacao'].mean().reset_index().set_index('filial').T", pandas_output=filial            A         B         C
avaliacao  7.027059  6.818072  7.072866, resposta_final="Com base nos resultados fornecidos pela consulta, podemos observar que a avaliação média de cada filial é a seguinte:\n\n- Filial A: 7,027059\n- Filial B: 6,818072\n- Filial C: 7,072866\n\nEsses valores indicam que a Filial C tem a avaliação média mais alta, seguida de perto pela Filial A, e a Filial B tem a avaliação média mais baixa entre as três.\n\nO código utilizado foi `df.groupby('filial')['avaliacao'].mean().reset_index().set_index('filial').T`")